In [ ]:
import numpy as np 
import pandas as pd 
import os

In [ ]:
import pandas as pd

In [ ]:
import imblearn
print(imblearn.__version__)

In [ ]:
df=pd.read_csv("./dataset/machine_predictive_maintenance.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.apply(lambda x: x.nunique())

In [ ]:
df.iloc[:,-1].unique()

# Covering Memory Leak
 ### Can not use both Target and Failure Type at the same time. As both of them together will have Data leak. 


In [ ]:
df.drop(df.columns[:2],axis=1,inplace=True)

In [ ]:
df.head()

# EDA

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.style.use('ggplot')

In [ ]:
sns.pairplot(df)

In [ ]:
plt.figure(figsize=(15,10))
for i,col in enumerate(df.columns,1):
    plt.subplot(3,3,i)
    sns.histplot(df[col],kde=True)

In [ ]:
from sklearn .preprocessing import LabelEncoder
encode=LabelEncoder()

In [ ]:
df.loc[:,df.columns[-1]]=encode.fit_transform(df.loc[:,df.columns[-1]])

In [ ]:
encode.classes_

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test = train_test_split(df,test_size = 0.3)

In [ ]:
df = X_train

In [ ]:
df.head()

## Need to min max Scaler on Roatation speed and Standard scaller on all
- ### Use MinMaxScaler as your default
- ### Use RobustScaler if you have outliers and can handle a larger range
- ### Use StandardScaler if you need normalized features
- ### Use Normalizer sparingly - it normalizes rows, not columns

In [ ]:
# Select only numeric columns for the correlation matrix
numeric_columns = df.select_dtypes(include=[np.number])

# Plot the heatmap of the correlation matrix
plt.figure(figsize=(15,10))
heatmap = sns.heatmap(numeric_columns.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix of Numeric Features')
plt.show()

In [ ]:
df.describe()

In [ ]:
df.drop(df.columns[-2],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df[df.columns[-1]].value_counts()

In [ ]:
df.Type.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df=pd.get_dummies(df,drop_first=True)

In [ ]:
X_test = pd.get_dummies(X_test,drop_first=True)

In [ ]:
X_test.head()

In [ ]:
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler

In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.columns=df.columns.astype("str")

In [ ]:
col_name=df.columns.to_list()

In [ ]:
col_name

In [ ]:
target_col=col_name.pop(-3)

In [ ]:
feature_col=col_name

In [ ]:
target_col

In [ ]:
feature_col

In [ ]:
y=df["Failure Type_1"].values

# Implementing Oversampling Due to Data Imbalance

In [ ]:
from imblearn.combine import SMOTETomek
from collections import Counter

In [ ]:
df_train=df.drop(df.columns[-3],axis=1)

In [ ]:
df_train_val=df_train.values

In [ ]:
smk=SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(df_train_val,y)

In [ ]:
X_res.shape,y_res.shape

In [ ]:
print(f"Orignal Dataset Shape {Counter(y)}")
print(f"Resampled Dataset Shape {Counter(y_res)}")

In [ ]:
df_new=pd.concat([pd.DataFrame(X_res),pd.DataFrame(y_res)],axis=1)

In [ ]:
df_new.head()

In [ ]:
col_new=df_train.columns
col_new=col_new.to_list()
col_new.append(target_col)
df_new.columns=col_new
df_new.head()

In [ ]:
#import plotly.plotly as py
! pip install chart-studio
from chart_studio import plotly

# ❔ Checking for outliers

In [ ]:
from plotly import __version__ 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
## Checking in Air Temp
mpl_fig = plt.figure(figsize=(10,5))

sns.boxplot(y=feature_col[0],x="Failure Type_1",data=df_new)

In [ ]:
## Checking in Process Temp
plt.figure(figsize=(10,5))
sns.boxplot(y=feature_col[1],x="Failure Type_1",data=df_new)

In [ ]:
## Checking in Rotational Speed

plt.figure(figsize=(10,5))
sns.boxplot(y=feature_col[2],x="Failure Type_1",data=df_new)

In [ ]:
## Checking in Torque
plt.figure(figsize=(10,5))
sns.boxplot(y=feature_col[3],x="Failure Type_1",data=df_new)


In [ ]:
## Checking in Tool wear
plt.figure(figsize=(10,5))
sns.boxplot(y=feature_col[4],x="Failure Type_1",data=df_new)

In [ ]:
sns.countplot(x="Failure Type_1",data=df_new)

In [ ]:
# As the data here is imbalanced we need to use 
y_res

# Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score
import warnings
warnings.filterwarnings('ignore')
acc_log=[]

# Using Naive Bayers

In [ ]:
X_test[target_col].values

In [ ]:
X_test.head(30)

In [ ]:
X_test[target_col].values

In [ ]:
from sklearn.naive_bayes import GaussianNB
acc_Gauss=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    
    clf=GaussianNB()
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    #print(X_train[target_col].values)
    print(y_pred)
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=roc_auc_score(y_valid,y_pred,multi_class="ovo")
#     acc_Gauss.append(acc)
#     print(f"The accuracy for {fold} : {acc}")
    
    pass

# Using SVM (Support Vector Machines)
### We need to choose the best Kernel according to our need.

- #### The linear kernel is mostly preferred for text classification problems as it performs well for large datasets. 
- #### Gaussian kernels tend to give good results when there is no additional information regarding data that is not available.
- #### Rbf kernel is also a kind of Gaussian kernel which projects the high dimensional data and then searches a linear separation for it.
- #### Polynomial kernels give good results for problems where all the training data is normalized. 


In [ ]:
from sklearn.svm import SVC
acc_svm=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    
    clf=SVC(kernel="linear")
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_svm.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

In [ ]:
from sklearn.svm import SVC
acc_svm_sig=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    
    clf=SVC(kernel="sigmoid")
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_valid)
    print(f"The fold is : {fold} : ")
    print(classification_report(y_valid,y_pred))
    
acc=accuracy_score(y_valid,y_pred)
acc_svm_sig.append(acc)
print(f"The accuracy for {fold} : {acc}")

In [ ]:
# from sklearn.svm import SVC
# acc_svm_rbf=[]
# kf=model_selection.StratifiedKFold(n_splits=5)
# for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
#     X_train=df_new.loc[trn_,feature_col]
#     y_train=df_new.loc[trn_,target_col]
    
#     X_valid=df_new.loc[val_,feature_col]
#     y_valid=df_new.loc[val_,target_col]
    
#     ro_scaler=RobustScaler()
#     X_train=ro_scaler.fit_transform(X_train)
#     X_valid=ro_scaler.transform(X_valid)
#     x_test_split=ro_scaler.transform(X_test[feature_col])
#     clf=SVC(kernel="rbf")
#     clf.fit(X_train,y_train)
#     y_pred=clf.predict(x_test_split)
#     print(f"The fold is : {fold} : ")
#     print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_svm_rbf.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

In [ ]:
from sklearn.svm import SVC
acc_svm_poly=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    clf=SVC(kernel="poly")
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_svm_poly.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

# Using  KNN
## The optimal K value usually found is the square root of N, where N is the total number of samples.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
acc_KNN=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    clf=KNeighborsClassifier(n_neighbors=4)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_KNN.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

# 🌲 Using Decission tree Classifier 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
acc_Dtree=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    clf=DecisionTreeClassifier(criterion="entropy")
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_Dtree.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

In [ ]:
# import time
# while True:
#     print("5")
#     time.sleep(60)

# 🚄 Using Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
acc_RandF=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    clf=RandomForestClassifier(n_estimators=55,criterion="entropy")
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_RandF.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

# 🚄 Using XGBoost

In [ ]:
class_failure=encode.classes_

In [ ]:
class_failure

In [ ]:
# Checking the best accuracy in K-SVM using which Kernel;
fold_list=["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5"]


In [ ]:
from xgboost import XGBClassifier
acc_XGB=[]
kf=model_selection.StratifiedKFold(n_splits=5)
for fold , (trn_,val_) in enumerate(kf.split(X=df_new,y=y_res)):
    
    X_train=df_new.loc[trn_,feature_col]
    y_train=df_new.loc[trn_,target_col]
    
    X_valid=df_new.loc[val_,feature_col]
    y_valid=df_new.loc[val_,target_col]
    
    ro_scaler=RobustScaler()
    X_train=ro_scaler.fit_transform(X_train)
    X_valid=ro_scaler.transform(X_valid)
    x_test_split=ro_scaler.transform(X_test[feature_col])
    clf=XGBClassifier()
    clf.fit(X_train,y_train)
    y_pred=clf.predict(x_test_split)
    print(f"The fold is : {fold} : ")
    print(classification_report(X_test[target_col].values,y_pred))
#     acc=accuracy_score(y_valid,y_pred)
#     acc_XGB.append(acc)
#     print(f"The accuracy for {fold} : {acc}")

In [ ]:
x_test_split

In [ ]:
import pickle
with open("XG_boost_pkl","wb") as files:
    pickle.dump(clf,files)

clf.save_model("model_sklearn.json")


# 🔚 Summary
### As we can see from the 97% AUC that we obtained using Random Forest as well as XGBoost